# Parte 2:  Validação Cruzada Para Seleção de Modelos

## Objetivos:

O objetivo desse desafio é compreender como funciona a validação cruzada e os benefícios em se utilizar essa técnica para selecionar modelos mais genéricos.


## Conceitos:

A validação cruzada é uma técnica de avaliação de um modelo em treinamento que permite estimar a capacidade de **generalização** do mesmo. A idéia por trás da técnica é validar qual seria o desempenho do modelo, treinado com uma parte dados, sobre o resto dos dados, nunca vistos antes pelo modelo. 

Essa técnica possibilita que se teste essa hipótese sem ser necessário ter uma base de testes extra, o que é muito útil quando se têm poucos dados. O diagrama abaixo resume a idéia central da técnica:


![Diagrama explicativo: Cross-Validation usando K-Fold com ${K = 10}$](images/k-fold-diagram.png)

A técnica mais comum, denominada `K-Fold`, consiste em particionar o dataset em $K$ grupos, treinar com $K-1$ grupos de dados e validar o modelo treinado sobre o grupo restante. Deve-se repetir esse passo $K$ vezes, para que todos os grupos sejam usados, e então calcular as métricas de avaliação como a média das métricas calculadas em cada iteração. Uma outra maneira de se fazer isso é armazenar a predição sobre todos os dados **quando em validação** e calcular as métricas de desempenho sobre essas predições.

O limite dessa técnica é o `Leave-One-Out`, caso é o extremo em que se particiona o dataset em $N$ grupos. Esse limite, quando possível, traz a informação mais acurada sobre a capacidade de generalização do modelo. Essa técnica tem como desvantagem a quantidade enorme de treinamentos realizados: um para cada elemento da base de treino.

___

# Imports

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import numpy as np
import os
import pandas as pd

In [ ]:
from sklearn.linear_model import ElasticNet
from sklearn.metrics.regression import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV, ParameterGrid, KFold
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

# Dataset:

## Sobre o Case

### Case baseado no dataset do Kaggle: "California Housing Prices"

Esse desafio é baseado em um dataset aberto do Kaggle ([https://www.kaggle.com](https://www.kaggle.com)) de 2018, de onde é possível estimar o preço de um imóvel pertencente a uma dada região na Califórnia. 

O dataset original foi extraído do repositório StatLib, que não está mais disponível. Os dados que o compôem foram retirados do Censo realizado na Califórnia em 1990 e modificado para servir como base de treinamento.


Link para o dataset no Kaggle: [https://www.kaggle.com/harrywang/housing/data](https://www.kaggle.com/harrywang/housing/data)


### Descrição dos Dados Originais:

#### Tamanho do Dataset:

* `20.640` data points

#### Variável dependente:

* `median_house_value`:  (float) variável dependente com o valor da mediana do preço de imóvel na região

#### Features: 

* `longitude`/`latitude`: (floats) posição global da região
* `housing_median_age`: (float) mediana da idade (em anos) das casas da região
* `total_rooms`: (float) total de aposentos da região
* `total_bedrooms`: (float) total de quartos da região
* `population`: (float) população total da região
* `households`: (float) quantidade total de imóveis da região
* `median_income`: (float) mediana do salário (por hora) de uma pessoa na região
* `ocean_proximity`: (string) categorias relativas à distância do oceano


### Modificação dos dados para o Desafio:

Para tornar o desafio mais fácil de avaliar, a massa de dados original foi dividida em duas massas, uma para treino e outra para teste, ambas contendo pouco mais de `10.000` elementos. 

Por motivos didáticos, alguns elementos da massa de treino foram removidos e, sobre os elementos restantes, foi aplicada uma Feature Engineering. Não é necesário se preocupar com o que é uma Feature Engineering agora, será o tema da Aula 20. 

## Carregando os dados

### Train Dataset 

In [ ]:
dataset = pd.read_csv("data/feature_engineered_california_housing_train.csv", sep="\t", index_col=0)
x_train = dataset.drop(["median_house_value"], axis=1)
y_train = dataset[["median_house_value"]]

In [ ]:
print(f"shape: {x_train.shape}")
x_train.head()

In [ ]:
print(f"shape: {y_train.shape}")
y_train.head()

### Test Dataset 

In [ ]:
dataset = pd.read_csv("data/feature_engineered_california_housing_test.csv", sep="\t", index_col=0)
x_test = dataset.drop(["median_house_value"], axis=1)
y_test = dataset[["median_house_value"]]

In [ ]:
print(f"shape: {x_test.shape}")
x_test.head()

In [ ]:
print(f"shape: {y_test.shape}")
y_test.head()

## Normalizando Features com Z-Score

Passo importante quando se treina modelos lineares, por eliminar importâncias artificialmente grandes para features contendo valores muito grandes.

In [ ]:
columns = x_train.columns

In [ ]:
zscore = StandardScaler().fit(x_train.loc[:, columns])

In [ ]:
x_train.loc[:, columns] = zscore.transform(x_train.loc[:, columns])
x_test.loc[:, columns] = zscore.transform(x_test.loc[:, columns])

# Problemas

## A) Seleção de Hiperparâmetros

O uso mais comum da técnica é a seleção dos `hiperparâmetros` que definem o modelo; 
diferente dos parâmetros do modelo, que são ajustados a cada iteração do treinamento, os hiperparâmetros são condições fixadas sob as quais o treinamento ocorre. Um exemplo de hiperparâmetro é o $alpha$, definido para treinamentos das regressões `Ridge` e `Lasso`.

Nessa seção, será feita a busca pelos melhores hiperparâmetros de treinamento usando a $ElasticNet$, que combina a regularização `L1` e `L2` em um modelo único. 

### Escolha dos Hiperparâmetros

Os hiperparâmetros a serem variados são:
* `alpha`: mesmo parâmetro visto em Regularização, define a intensidade da regularização no modelo.
* `l1_ratio `: define o tipo de regularização, como mostrado abaixo:
    * $l1\_ratio = 1$: o treinamento ocorre com `L1` apenas
    * $l1\_ratio = 0$: o treinamento ocorre com `L2` apenas
    * $0 \leq l1\_ratio \leq 1$: o treinamento ocorre com uma combinação de `L1` com `L2` na proporção dada.

#### Variação dos hiperparâmetros

In [ ]:
# hiperparâmetros
param_grid = {
    "alpha": np.logspace(1, 4, 11),          # Por quê `logspace`?
    "l1_ratio": np.linspace(0.01, 1.0, 11),
    "max_iter": [100],
    "positive": [True]
}

##### Treinamento dos modelos

Usar o [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) para treinar um conjunto de modelos com a combinação de todos os hiperparâmetros.

Parâmetros a serem utilizados:

* `estimator`: [`ElasticNet()`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html)
* `param_grid `: `param_grid`
* `scoring`: 'r2'
* `cv`: usar [`KFold(n_splits=10, shuffle=True, random_state=42)`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html)
* `random_state`: 42

In [ ]:
""" Escreva a solução aqui """

####  Avaliação dos modelos treinados

Mostrar uma tabela contendo todos os treinamentos e suas avaliações e escolher um range de exploração onde o modelo consegue ter uma performance melhor.

Dica: usar a função [`heatmap`](https://seaborn.pydata.org/generated/seaborn.heatmap.html) do `seaborn` para visualizar os melhores parâmetros.

In [ ]:
""" Escreva a solução aqui """

## B) Estudo da influência do parâmetro $K$ sobre a generalização

Quanto mais `folds` forem usados na validação cruzada, mais realista será a estimativa do poder de generalização do modelo treinado. Como já foi citado, o uso de `LeaveOneOut` é o mais próximo de trazer essa estimativa, com o contraponto de ser o mais pesado computacionalmente.

Nessa seção, será feita uma comparação entre o desempenho dos modelos nas massas de validação (calculado com dados de treino durante a validação cruzada) e de teste. 

###### Importante:

Os hipeparâmetros a serem variados nos treinamentos devem ser obtidos na última etapa do item **A**.

####  Hiperparâmetros

Preencher `params_grid` com as faixas de hiperparâmetros de melhor performance.

In [ ]:
""" Complete os espaços com ? """
param_grid = {
    "alpha": ?,
    "l1_ratio": ?,
    "max_iter": [1000],
    "positive": [True]
}

####  Treinar os modelos para os valores de $K$

Treinar os modelos para cada valor de $K$ em `k_list`. Armazenar, para cada $K$, os valores de $R^2$ médio calculados sobre a massa de **treino** e de **validação** (testes dentro do *Cross Validation*) e o $R^2$ calculado sobre a massa de **testes**.

In [ ]:
k_list = [3, 5, 10, 20, 35, 60, 100]

In [ ]:
""" Escreva a solução aqui """

####  Avaliação da generalização

Mostrar em um plot a comparação entre os valores de $R^2$ calculados sobre as massa de treino, teste e validação para cada variação de $K$. 

* Eixo X: valores de $K$
* Eixo Y: valores de $R^2$


In [ ]:
""" Escreva a solução aqui """